# Part 5: Intro to Parallelization

This part will briefly cover how to run GRASS computations in parallel. It requires computing the first two parts of the workshop. 

First, create a new mapset:

In [ ]:
%%bash
grass -c -e ~/grassdata/dix_park/parallelization

In [ ]:
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("~/grassdata", "dix_park", "parallelization")

## Tool-level parallelization
There are several [internally parallelized tools](https://grass.osgeo.org/grass-stable/manuals/keywords.html#parallel), either using OpenMP or Python multiprocessing library. We can use `nprocs` option to set the number of cores to be used for processing.


Set computational region to match dsm raster.

In [ ]:
gs.run_command("g.region", raster="dsm")

Compute moving window analysis and measure time first with one core, then with 2:

In [ ]:
%%timeit -n1 -r1
gs.run_command("r.neighbors", input="dsm", output="dsm_smoothed", method="average", size=25, nprocs=1)

In [ ]:
%%timeit -n1 -r1
gs.run_command("r.neighbors", input="dsm", output="dsm_smoothed", method="average", size=25, nprocs=2)

Visualize original and smoothed raster (turn layers on and off):

In [ ]:
neighbors_map = gj.InteractiveMap()
neighbors_map.add_raster("dsm")
neighbors_map.add_raster("dsm_smoothed")
neighbors_map.add_layer_control(position="bottomright")
neighbors_map.show()

## GridModule for tiling
Some compute-intensive tasks can benefit from spatially splitting the task into tiles, and then running the task in parallel. [GridModule](https://grass.osgeo.org/grass-stable/manuals/libpython/pygrass.modules.grid.html) can automate this splitting-computing-merging procedure and execute the computation in parallel.

In this example, we will interpolate an elevation surface from vector points using IDW interpolation. First, generate points randomly by sampling the digital elevation model. We adjusted the resolution to run the examples faster.

In [ ]:
gs.run_command("g.region", res=2)
gs.run_command("r.random", flags="z", input="ground", npoints=5000, vector="samples", seed=1, quiet=True)

Measure the time without using GridModule:

In [ ]:
%%timeit -n1 -r1
gs.run_command("v.surf.idw", input="samples", output="idw")

And now with GridModule:

In [ ]:
%%writefile interpolation.py
from grass.pygrass.modules.grid import GridModule


grid = GridModule(
    "v.surf.idw",
    input="samples",
    output="idw",
    processes=3,
    overlap=20,
    quiet=True,
)
grid.run()

In [ ]:
%%timeit -n1 -r1
!python interpolation.py

In [ ]:
neighbors_map = gj.Map()
neighbors_map.d_rast(map="idw")
neighbors_map.d_vect(map="samples", size=1, color="black")
neighbors_map.show()

## Running multiple independent computations
In this example, our goal is to compute multiple viewsheds and render them into a PNG file.
Since these are independent computations, we can run them in parallel.
The first part implements this task in Python using _multiprocessing_ library
and the second part will run each computation using `grass --exec` interface in separate mapsets that allows us to potentially distribute the computation across multiple nodes on an HPC.

First compute a shaded relief raster for visualization:

In [ ]:
gs.run_command("g.region", raster="dsm")
gs.run_command("r.relief", input="dsm", output="relief")

We will use the viewshed points form part 2:

In [ ]:
viewpoints = gs.read_command('v.out.ascii', input='viewpoints@viewshed',
                             separator='comma', layer=2, where="height < 2").strip().splitlines()
viewpoints = [p.split(",") for p in viewpoints]

We will run viewshed computation from part 2 using `multiprocessing.Pool`. We define a function that computes the viewshed and returns the name of the output or None in case of error:

In [ ]:
%%time
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name,
                       coordinates=(x, y), max_distance=300, flags="b")
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 2
with Pool(processes=proc) as pool:
    maps = pool.map(viewshed, viewpoints)
print(maps)

One trick to speedup viewshed computation is to limit the computation only
to the actual area given by the maxdistance option. To do that we will locally modify the computational region
and pass the environment to the module directly. The current computational region won't be affected. 
Additionally, this shows how to include a simple progress bar.

In [ ]:
%%time
import os
from tqdm import tqdm
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # set GRASS_REGION variable using region_env function
    os.environ["GRASS_REGION"] = gs.region_env(align="dsm",
                                               e=x + max_distance,
                                               w=x - max_distance,
                                               n=y + max_distance,
                                               s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name, flags="b",
                      coordinates=(x, y), max_distance=max_distance)
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
proc = 2
with Pool(processes=proc) as pool:
    maps = list(tqdm(pool.imap(viewshed, viewpoints), total=len(viewpoints)))
print(maps)
print(f"Viewshed num cells: {gs.raster_info(maps[0])['cells']}")
print(f"DSM num cells: {gs.raster_info('dsm')['cells']}")

Now we extend this script to include rendering to file:

In [ ]:
import os
from grass.exceptions import CalledModuleError
from multiprocessing import Pool, cpu_count


def viewshed(point):
    x, y, cat = point
    x, y = float(x), float(y)
    max_distance = 300
    # set GRASS_REGION variable using region_env function
    os.environ["GRASS_REGION"] = gs.region_env(align="dsm",
                                               e=x + max_distance,
                                               w=x - max_distance,
                                               n=y + max_distance,
                                               s=y - max_distance)
    name = f"viewshed_{cat}"
    try:
        gs.run_command("r.viewshed", input="dsm", output=name, flags="b",
                      coordinates=(x, y), max_distance=max_distance)
        # create visualization
        viewshed_map = gj.Map(use_region=True)
        viewshed_map.d_rast(map="relief")
        viewshed_map.d_rast(map=f"viewshed_{cat}", values=1)
        viewshed_map.d_vect(map="viewpoints@viewshed", layer=2, cat=cat, size=15, icon="basic/pin")
        viewshed_map.save(f"viewshed_{cat}.png")
        return f"viewshed_{cat}"
    except CalledModuleError:
        return None

# run with the number of CPUs available
# proc = cpu_count()
nprocs = 2
with Pool(processes=nprocs) as pool:
    maps = list(tqdm(pool.imap(viewshed, viewpoints), total=len(viewpoints)))

Let's look at one of the computed and rendered viewsheds:

In [ ]:
from IPython.display import Image

Image("viewshed_15.png")

Note that this way, we can't distribute the computation across multiple nodes (hundreds of cores).
We will do the same thing differently, using `grass --exec` [interface](https://grass.osgeo.org/grass-stable/manuals/grass.html), running each task in a separate mapset. This way, the tasks could be distributed across multiple nodes.

`--exec` interface allows GRASS tools and user scripts to be executed in a GRASS non-interactive session. For example, here is a simple call to list all available vectors in PERMANENT mapset:

In [ ]:
%%bash
grass ~/grassdata/dix_park/PERMANENT --exec g.list type=vector mapset=viewshed -t

Now we will create a Python script `myscript.py` computing and rendering viewsheds similarly as in the previous example. The script requires 3 parameters (x and y coordinate, and category). Note that we can set computational region in a standard way, because each script will run in separate mapset, so the different regions won't interfere with each other.

In [ ]:
%%writefile myscript.py
import sys
import grass.script as gs
import grass.jupyter as gj


def main(x, y, cat):
    max_distance = 300
    x, y = float(x), float(y)
    name = f"viewshed_{cat}"
    gs.run_command("g.region", align="dsm", e=x + max_distance,
                   w=x - max_distance, n=y + max_distance, s=y - max_distance)
    gs.run_command("r.viewshed", input="dsm", output=name, coordinates=(x, y),
                   observer_elevation=3, max_distance=max_distance, flags="b")
    # create visualization
    viewshed_map = gj.Map(use_region=True)
    viewshed_map.d_rast(map="relief@parallelization")
    viewshed_map.d_rast(map=f"viewshed_{cat}", values=1)
    viewshed_map.d_vect(map="viewpoints@viewshed", layer=2, cat=cat, size=15, icon="basic/pin")
    viewshed_map.save(f"viewshed_{cat}.png")

if __name__ == "__main__":
    args = sys.argv[1:]
    main(*args)

We will generate a file `jobs.sh` with one command per line. We will run each task in a temporary mapset so all computed data will be deleted afterwards. That is fine for our example where we need only the final PNG files.

In [ ]:
with open("jobs.sh", "w") as f:
    for viewpoint in viewpoints:
        f.write(f"grass --tmp-mapset ~/grassdata/dix_park --exec python myscript.py {viewpoint[0]} {viewpoint[1]} {viewpoint[2]}\n")

This is the content of the file:

In [ ]:
!cat jobs.sh

To execute these commands in parallel, we can use e.g. [GNU Parallel](https://www.gnu.org/software/parallel/):

In [ ]:
%%bash

parallel -j 2 < jobs.sh

Check one of the resulting PNG files:

In [ ]:
Image("viewshed_15.png")